<a href="https://colab.research.google.com/github/JeongYoon-L/Analysis-TMDB-open-data/blob/main/cse351_project_Lee_JeongYoon_114133199.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#JeongYoon Lee(114133199)_CSE351_Summer_project

- Project #1: Movie Revenue Prediction

In [22]:
#import libraries

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt


In [73]:
#read csv file
movies_original = pd.read_csv('tmdb_5000_movies.csv')
credits_original = pd.read_csv('tmdb_5000_credits.csv')
from ast import literal_eval


In [24]:
#check shape
movies_original.shape

(4803, 20)

In [6]:
#check shape
credits_original.shape

(4803, 4)

In [25]:
#check duplicated id
display(movies_original[movies_original.duplicated("id")])

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [30]:
#check duplicated id
display(credits_original[credits_original.duplicated("movie_id")])

,movie_id,title,cast,crew


In [82]:
credits_original.rename(columns = {'movie_id' : 'id'}, inplace = True)
credits_original = credits_original.drop(columns = 'title')
credits_original.head()


,id,cast,crew
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [101]:
#merge(left join) with two csv file
data = pd.merge(left = movies_original, right = credits_original, how = "left" , on = "id")
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [37]:
#check the number of null value for each columns.
data.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
dtype: int64

In [102]:
#fill "NaN" to null value in homepage, overview, tagline columns
data["homepage"] = data["homepage"].fillna("NaN")
data["overview"] = data["overview"].fillna("NaN")
data["tagline"] = data["tagline"].fillna("NaN")

#check the number of null value for each columns.
data.isnull().sum()

budget                  0
genres                  0
homepage                0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            1
revenue                 0
runtime                 2
spoken_languages        0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
cast                    0
crew                    0
dtype: int64

In [103]:
data[data["release_date"].isnull()]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
4553,0,[],NaN,380097,[],en,America Is Still the Place,1971 post civil rights San Francisco seemed li...,0.0,[],[],NaN,0,0.0,[],Released,NaN,America Is Still the Place,0.0,0,[],[]


When I searched this movie to the "themoviedb.org", and there's no such data for this movie.
So I decided remove this row since there are no any information about this movie in the website also any columns.


In [104]:
drop_row = data[data['release_date'].isnull()].index
data = data.drop( drop_row )
data[data["release_date"].isnull()]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew


In [105]:
data[data["runtime"].isnull()]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
2656,15000000,"[{""id"": 18, ""name"": ""Drama""}]",NaN,370980,"[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...",it,Chiamatemi Francesco - Il Papa della gente,NaN,0.738646,"[{""name"": ""Taodue Film"", ""id"": 45724}]","[{""iso_3166_1"": ""IT"", ""name"": ""Italy""}]",2015-12-03,0,NaN,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,NaN,Chiamatemi Francesco - Il Papa della gente,7.3,12,"[{""cast_id"": 5, ""character"": ""Jorge Mario Berg...","[{""credit_id"": ""5660019ac3a36875f100252b"", ""de..."
4140,2,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,459488,"[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...",en,"To Be Frank, Sinatra at 100",NaN,0.050625,"[{""name"": ""Eyeline Entertainment"", ""id"": 60343}]","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2015-12-12,0,NaN,[],Released,NaN,"To Be Frank, Sinatra at 100",0.0,0,"[{""cast_id"": 0, ""character"": ""Narrator"", ""cred...","[{""credit_id"": ""592b25e4c3a368783e065a2f"", ""de..."


For the movie "Chiamatemi Francesco - Il Papa della gente" which its runtime is null value, we can find that the real runtime for this movie is 1h38m through the tmdb website.
https://www.themoviedb.org/movie/370980-chiamatemi-francesco


In [106]:
data.at[2656, 'runtime'] = 98
data.loc[2656]

budget                                                           15000000
genres                                      [{"id": 18, "name": "Drama"}]
homepage                                                              NaN
id                                                                 370980
keywords                [{"id": 717, "name": "pope"}, {"id": 5565, "na...
original_language                                                      it
original_title                 Chiamatemi Francesco - Il Papa della gente
overview                                                              NaN
popularity                                                       0.738646
production_companies               [{"name": "Taodue Film", "id": 45724}]
production_countries              [{"iso_3166_1": "IT", "name": "Italy"}]
release_date                                                   2015-12-03
revenue                                                                 0
runtime                               

In [107]:
data[data["runtime"].isnull()]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
4140,2,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,459488,"[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...",en,"To Be Frank, Sinatra at 100",NaN,0.050625,"[{""name"": ""Eyeline Entertainment"", ""id"": 60343}]","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2015-12-12,0,NaN,[],Released,NaN,"To Be Frank, Sinatra at 100",0.0,0,"[{""cast_id"": 0, ""character"": ""Narrator"", ""cred...","[{""credit_id"": ""592b25e4c3a368783e065a2f"", ""de..."


For the movie "To Be Frank, Sinatra at 100" which its runtime is null value, we can find that the real runtime for this movie is 1h21m through the tmdb website. https://www.themoviedb.org/movie/459488-to-be-frank-sinatra-at-100

In [108]:
data.at[4140, 'runtime'] = 81
data.loc[4140]

budget                                                                  2
genres                                [{"id": 99, "name": "Documentary"}]
homepage                                                              NaN
id                                                                 459488
keywords                [{"id": 6027, "name": "music"}, {"id": 225822,...
original_language                                                      en
original_title                                To Be Frank, Sinatra at 100
overview                                                              NaN
popularity                                                       0.050625
production_companies     [{"name": "Eyeline Entertainment", "id": 60343}]
production_countries     [{"iso_3166_1": "GB", "name": "United Kingdom"}]
release_date                                                   2015-12-12
revenue                                                                 0
runtime                               

In [109]:
#make dict to list
data['genres']=data['genres'].apply(literal_eval)
data['genres']=data['genres'].apply(lambda x : [y['name'] for y in x] )

data['keywords']=data['keywords'].apply(literal_eval)
data['keywords']=data['keywords'].apply(lambda x : [y['name'] for y in x] )

data['production_companies']=data['production_companies'].apply(literal_eval)
data['production_companies']=data['production_companies'].apply(lambda x : [y['name'] for y in x] )

data['production_countries']=data['production_countries'].apply(literal_eval)
data['production_countries']=data['production_countries'].apply(lambda x : [y['name'] for y in x] )

data['spoken_languages']=data['spoken_languages'].apply(literal_eval)
data['spoken_languages']=data['spoken_languages'].apply(lambda x : [y['iso_639_1'] for y in x] )

data['cast']=data['cast'].apply(literal_eval)
data['cast']=data['cast'].apply(lambda x : [y['name'] for y in x] )

data['crew']=data['crew'].apply(literal_eval)
data['crew']=data['crew'].apply(lambda x : [y['name'] for y in x] )

In [110]:
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[en, es]",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,[en],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe..."
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,"[fr, en, es, it, de]",Released,A Plan No One Escapes,Spectre,6.3,4466,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John..."
3,250000000,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,[en],Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Hans Zimmer, Charles Roven, Christopher Nolan..."
4,260000000,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,[en],Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Andrew Stanton, Andrew Stanton, John Lasseter..."


In [111]:
data['num_of_genres'] = data["genres"].apply(lambda x: len(x) if x != {} else 0).value_counts()
data['num_of_keywords'] = data["keywords"].apply(lambda x: len(x) if x != {} else 0).value_counts()
data['num_of_production_companies'] = data["production_companies"].apply(lambda x: len(x) if x != {} else 0).value_counts()
data['num_of_production_countries'] = data["production_countries"].apply(lambda x: len(x) if x != {} else 0).value_counts()
data['num_of_spoken_languages'] = data["spoken_languages"].apply(lambda x: len(x) if x != {} else 0).value_counts()
data['num_of_cast'] = data["cast"].apply(lambda x: len(x) if x != {} else 0).value_counts()
data['num_of_crew'] = data["crew"].apply(lambda x: len(x) if x != {} else 0).value_counts()


data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,num_of_genres,num_of_keywords,num_of_production_companies,num_of_production_countries,num_of_spoken_languages,num_of_cast,num_of_crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[en, es]",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B...",27.0,411.0,350.0,173.0,85.0,42.0,27.0
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,[en],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe...",900.0,288.0,1066.0,3383.0,3355.0,10.0,137.0
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,"[fr, en, es, it, de]",Released,A Plan No One Escapes,Spectre,6.3,4466,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John...",1490.0,283.0,1062.0,851.0,841.0,10.0,261.0
3,250000000,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,[en],Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Hans Zimmer, Charles Roven, Christopher Nolan...",1525.0,344.0,916.0,275.0,308.0,30.0,223.0
4,260000000,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,[en],Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Andrew Stanton, Andrew Stanton, John Lasseter...",636.0,387.0,587.0,93.0,133.0,76.0,144.0


- divide DataFrame into two dataframes.
- One for continous variables with revenue, and the other for catagorical variables with revenue.

In [120]:
data_continuous = data[['id','revenue','budget','popularity','release_date','runtime','vote_average','vote_count','num_of_genres', 'num_of_keywords','num_of_production_companies','num_of_production_countries','num_of_spoken_languages','num_of_cast','num_of_crew' ]]
data_continuous.head()

,id,revenue,budget,popularity,release_date,runtime,vote_average,vote_count,num_of_genres,num_of_keywords,num_of_production_companies,num_of_production_countries,num_of_spoken_languages,num_of_cast,num_of_crew
0,19995,2787965087,237000000,150.437577,2009-12-10,162.0,7.2,11800,27.0,411.0,350.0,173.0,85.0,42.0,27.0
1,285,961000000,300000000,139.082615,2007-05-19,169.0,6.9,4500,900.0,288.0,1066.0,3383.0,3355.0,10.0,137.0
2,206647,880674609,245000000,107.376788,2015-10-26,148.0,6.3,4466,1490.0,283.0,1062.0,851.0,841.0,10.0,261.0
3,49026,1084939099,250000000,112.312950,2012-07-16,165.0,7.6,9106,1525.0,344.0,916.0,275.0,308.0,30.0,223.0
4,49529,284139100,260000000,43.926995,2012-03-07,132.0,6.1,2124,636.0,387.0,587.0,93.0,133.0,76.0,144.0


In [122]:
data_categorical = data[['id','revenue','genres','homepage','keywords','original_language','original_title','overview','production_companies','production_countries','spoken_languages','status','tagline','title','cast','crew']]
data_categorical.head()

,id,revenue,genres,homepage,keywords,original_language,original_title,overview,production_companies,production_countries,spoken_languages,status,tagline,title,cast,crew
0,19995,2787965087,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]","[en, es]",Released,Enter the World of Pandora.,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."
1,285,961000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],[en],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe..."
2,206647,880674609,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]","[fr, en, es, it, de]",Released,A Plan No One Escapes,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John..."
3,49026,1084939099,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],[en],Released,The Legend Ends,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[Hans Zimmer, Charles Roven, Christopher Nolan..."
4,49529,284139100,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",[Walt Disney Pictures],[United States of America],[en],Released,"Lost in our world, found in another.",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Andrew Stanton, Andrew Stanton, John Lasseter..."
